In [6]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm
from nasbench import api
from search_spaces import load_nasbench_101, load_proxylessnas, load_nasbench_nlp
from random_search import run_random_search, random_spec
from neural_predictor import classifier, regressor
from input_preprocessing import preprocess_nasbench, preprocess_nasbench_nlp, preprocess_proxylessnas

from scipy.stats import kendalltau as tau
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

### NAS-Bench-101

In [3]:
nasbench = load_nasbench_101()

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 57 seconds


In [4]:
def get_N_samples(N):
    models = []

    for _ in range(N):
        while True:
            model = random_spec(nasbench)
            if model not in models:
                models.append(nasbench.query(model))
                break
                
    return preprocess_nasbench(models)

In [5]:
models = get_N_samples(119+100000)

In [7]:
reg = regressor([models['X'][:79], models['norm_A'][:79], models['norm_AT'][:79]], models['val_acc'][:79])

In [11]:
y_pred = reg.predict([models['X'][:79], models['norm_A'][:79], models['norm_AT'][:79]])
y_true = models['val_acc'][:79]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

0.6586282422977711 0.7770186476462595 0.9533437131864096


In [12]:
np.save('outputs/nasbench_y_true_train.npy', y_true)
np.save('outputs/nasbench_y_pred_train.npy', y_pred)
np.save('outputs/nasbench_mse_train.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nasbench_tau_train.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nasbench_r2_train.npy', r2_score(y_true, y_pred))

In [13]:
y_pred = reg.predict([models['X'][79:119], models['norm_A'][79:119], models['norm_AT'][79:119]])
y_true = models['val_acc'][79:119]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred))

2.5534242721048512 0.5869484780941913 0.8353852262596422


In [14]:
np.save('outputs/nasbench_y_true_val.npy', y_true)
np.save('outputs/nasbench_y_pred_val.npy', y_pred)
np.save('outputs/nasbench_mse_val.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nasbench_tau_val.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nasbench_r2_val.npy', r2_score(y_true, y_pred))

In [15]:
y_pred = reg.predict([models['X'][119:], models['norm_A'][119:], models['norm_AT'][119:]])
y_true = models['val_acc'][119:]

In [16]:
min_idx = np.argmin(y_pred.ravel())
max_idx = np.argmax(y_pred.ravel())

In [17]:
y_idx = np.where(y_pred > y_pred[min_idx])[0]

In [18]:
y_pred_10 = np.vstack([y_pred[min_idx], y_pred[list(range(0, len(y_idx), len(y_idx) // 8))[-8:]], y_pred[max_idx]]).ravel()
y_true_10 = np.hstack([y_true[min_idx], y_true[list(range(0, len(y_idx), len(y_idx) // 8))[-8:]], y_true[max_idx]]).ravel()
print(mean_squared_error(y_true_10, y_pred_10), tau(y_true_10, y_pred_10).correlation, r2_score(y_true_10, y_pred_10))

5.098841614816604 0.5111111111111111 0.8584448884306847


In [19]:
np.save('outputs/nasbench_y_true_test.npy', y_true)
np.save('outputs/nasbench_y_pred_test.npy', y_pred)
np.save('outputs/nasbench_y_true_test_10.npy', y_true_10)
np.save('outputs/nasbench_y_pred_test_10.npy', y_pred_10)
np.save('outputs/nasbench_mse_test.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nasbench_tau_test.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nasbench_r2_test.npy', r2_score(y_true, y_pred))

### ProxylessNAS

In [ ]:
for i, path in enumerate(['model_acc_cifar100', 'model_acc_caltech', 'model_acc_oxford_pet']):
    train_data = 
    valid_data =
    test_data = 
# reg = regressor(train_data, labels, N=N, n_gcn=18, n_hidden_fc=[512, 128], init_lr=0.001, weight_decay=0.00001)

### NAS-Bench-NLP

In [ ]:
N = 172

In [20]:
search_set, env = load_nasbench_nlp()

In [ ]:
def get_N_nlp_samples(N):
    env.reset()
    
    models = []
    stats = []

    for _ in range(N):
        while True:
            idx = np.random.choice(len(search_set), 1, replace=False)[0]
            model = search_set[idx]
            
            if model not in models:
                env.simulated_train(model, 50)
                
                if 'OK' == env.get_model_status(model):   
                    models.append(model)
                    stats.append(env.get_model_stats(model, 49))
                    break
                
    return preprocess_nasbench_nlp(models, stats)

In [ ]:
# training part
train_data = get_N_nlp_samples(119+10000)
reg = regressor([train_data['X'], train_data['norm_A'], train_data['norm_AT']], train_data['val_loss'], mode='loss')